In [2]:
import transformers
import torch

In [3]:
gpt_tokenizer = transformers.GPT2Tokenizer.from_pretrained('gpt2')
gpt_model = transformers.GPT2LMHeadModel.from_pretrained('gpt2')

In [4]:
from src.GPT2 import GPT2

model=GPT2(gpt_tokenizer)

model.load_from_original(gpt_model)


In [16]:
x=gpt_tokenizer.encode("Hello world, i have been away for too long", return_tensors="pt")
edge_index=torch.randint(0,x.shape[-1]-1,(2,8))

x

edge_index

model(x,edge_index)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (10x768 and 3072x768)

torch.Size([1, 10])